In [1]:
import sys
import os

# Adiciona o diretório raiz ao path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
from utils.text import normaliza_texto

import pandas as pd

[nltk_data] Downloading package stopwords to /home/esdras-
[nltk_data]     daniel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_csv('/home/esdras-daniel/Documentos/Python/PGM-DataAnalysis/data/raw/pgm-dataset.csv')

In [3]:
df.head(3)

,teorTexto,idAviso,setorDestino,dataAviso,tipoAviso,orgaoJulgador,assuntos,documentos,anexos,classeProcesso
0,PODER JUDICIÁRIO ESTADO DO RIO GRANDE DO NORTE...,18612632.0,Procuradoria da Sa�de,2024-06-21 05:00:40,IntimaÆo,2ª Vara da Infância e Juventude da Comarca de ...,12485;12494,Decisão;Petição de manifestação;Despacho;Certi...,notaTecnica-228951.pdf - notaTecnica-228951;E-...,1706
1,PODER JUDICIÁRIO DO ESTADO DO RIO GRANDE DO NO...,18629258.0,Procuradoria Administrativa,2024-06-25 23:59:59,IntimaÆo,3º Juizado da Fazenda Pública da Comarca de Natal,10715,Notificação;Petição;Despacho;Diligência;Notifi...,FICHA FINANCEIRA;SEMAD - Joelma;08464942120228...,14695
2,PODER JUDICIÁRIO DO ESTADO DO RIO GRANDE DO NO...,18629422.0,CARTORIO,2024-06-18 09:27:16,IntimaÆo,3ª Vara de Execução Fiscal e Tributária de Natal,10536;5952,Certidão;Certidão;Sentença;Petição;Intimação;C...,"0895416-93.2022, AR;Certidão de Dívida Ativa;C...",1116


# 1. Substituindo valores nas colunas

## 1.1 - setorDestino

In [4]:
setores_mapeamento = {
    'Procuradoria Administrativa': 'Administrativa',
    'Chefia Administrativa': 'Administrativa',
    'SETOR DE ADMINISTRA�?O GERAL': 'Administrativa',

    'Procuradoria Fiscal': 'Fiscal',
    'APOIO FISCAL': 'Fiscal',
    'CHEFIA FISCAL': 'Fiscal',

    'Procuradoria da Sa�de': 'Saúde',

    'Procuradoria Judicial': 'Judicial',
    'Chefia Judicial': 'Judicial',

    'Procuradoria do Meio Ambiente': 'Meio Ambiente',
    'Chefia Meio Ambiente': 'Meio Ambiente',

    'Chefia Patrimonial': 'Patrimonial',

    'DEPARTAMENTO DE C�LCULOS E CONTABILIDADE': 'Contabilidade',
    
    'CARTORIO': 'Cartório',
    'Assessoria - Gabinete': 'Cartório',
    'Procuradoria Geral': 'Cartório',
    'Procuradoria Geral Substituta': 'Cartório',

    'Procuradoria de Recupera�?o da D�vida Ativa': 'PRDA'
}

In [5]:
# Substituindo os valores de 'setorDestino'
#df['setorDestino'].replace(setores_mapeamento, inplace=True)
df.replace({'setorDestino': setores_mapeamento}, inplace=True)

In [34]:
df['setorDestino'].value_counts()

setorDestino
Fiscal            6686
Administrativa    5403
Contabilidade     1338
Judicial          1137
Saúde              443
Meio Ambiente      155
Patrimonial        121
Name: count, dtype: int64

## 1.2 - tipoAviso

In [14]:
swap_erros = {'IntimaÆo': 'Intimação',
              'CitaÆo': 'Citação',
              'audincia': 'audiência',
              'NotificaÆo': 'Notificação',
              'Intima��o': 'Intimação'}

df.replace({'tipoAviso': swap_erros}, regex=True, inplace=True)
#df['tipoAviso'].replace(swap_erros, regex=True, inplace=True)

In [15]:
df['tipoAviso'].value_counts()

tipoAviso
Intimação                 13632
Intimação de Pauta         1254
Citação                    1024
Notificação                  25
Intimação de audiência        7
Name: count, dtype: int64

## 1.3 - Orgão Julgador

In [16]:
abreviacoes_orgaos = {
    "Vara de Execução Fiscal e Tributária": "VEFT",
    "Vara da Fazenda Pública": "VFP",
    "Vara de Família e Sucessões": "VFS",
    "Vara da Infância e Juventude": "VIJ",
    "Vara Cível": "VC",
    "Juizado da Fazenda Pública": "JFP",
    "Juizado Especial Cível, Criminal e da Fazenda Pública": "JCCFP",
    "Turma Recursal": "TR",
    "Gabinete do Desembargador": "Gab. Des.",
    "Gabinete da Juíza": "Gab. Juíza",
    "Gabinete do Juiz": "Gab. Juiz",
    "Gabinete da Vice-Presidência": "Gab. VP",
    "Comarca de": 'Com.',
    "Juizado Especial da Fazenda Pública": "JEFP"
}


In [17]:
df.replace({'orgaoJulgador': abreviacoes_orgaos}, regex=True, inplace=True)

# 2. Removendo colunas e linhas

In [18]:
# Removendo colunas
df.drop(columns=['idAviso', 'dataAviso'], inplace=True)

## 2.1 - Removendo valores nulos

In [19]:
# Valores nulos de 'setorDestino'
df = df[~df['setorDestino'].isna()]

In [20]:
# Trando valores nulos das outras colunas
df.fillna({'anexos': 'Sem Anexos',
           'documentos': 'Sem Documentos',
           'assuntos': 'Sem Assuntos'}, inplace=True)

## 2.2 - Removendo as classes PRDA e Cartório

* Motivo de remoção do cartório -> É o setor responsável pela distribuição dos processos
* Motivo de remoção do PRDA -> Pouquissímos exemplos

In [29]:
df = df[~df['setorDestino'].isin(['Cartório', 'PRDA'])]

# 3. Salvando o DF

In [31]:
nome_df = 'pgm-dataset-processado.csv'
raw_processed = 'processed'

df.to_csv(f'/home/esdras-daniel/Documentos/Python/PGM-DataAnalysis/data/{raw_processed}/{nome_df}', index=False)

In [32]:
df

,teorTexto,setorDestino,tipoAviso,orgaoJulgador,assuntos,documentos,anexos,classeProcesso
0,PODER JUDICIÁRIO ESTADO DO RIO GRANDE DO NORTE...,Saúde,Intimação,2ª VIJ da Com. Natal,12485;12494,Decisão;Petição de manifestação;Despacho;Certi...,notaTecnica-228951.pdf - notaTecnica-228951;E-...,1706
1,PODER JUDICIÁRIO DO ESTADO DO RIO GRANDE DO NO...,Administrativa,Intimação,3º JFP da Com. Natal,10715,Notificação;Petição;Despacho;Diligência;Notifi...,FICHA FINANCEIRA;SEMAD - Joelma;08464942120228...,14695
3,PODER JUDICIÁRIO DO ESTADO DO RIO GRANDE DO NO...,Fiscal,Intimação,5ª VEFT de Natal,9518,Despacho;Petição;Despacho;Petição;Despacho;Pet...,docs habilitação;Sisbajud Positivo;CÁLCULOS;Ex...,156
4,PODER JUDICIÁRIO DO ESTADO DO RIO GRANDE DO NO...,Fiscal,Intimação,1ª VEFT de Natal,10534;5952,Despacho;Certidão;Certidão;Petição;Decisão;Pet...,ComprovanteValidacao - 2022-DP-PREC-47082;cda;...,12078
5,PODER JUDICIÁRIO DO ESTADO DO RIO GRANDE DO NO...,Administrativa,Intimação,5ª VFP da Com. Natal,10311,Petição;Decisão;Petição;Sentença;Petição;Decis...,calculos (1);LEI;justificativa;FICHA FINANCEIR...,12078
...,...,...,...,...,...,...,...,...
15935,Poder Judiciário do Rio Grande do Norte3ª Turm...,Fiscal,Intimação de Pauta,6º JFP da Com. Natal,6062,Contrarrazões;Ato Ordinatório;Recurso Inominad...,Contrarrazões;FICHAS FINANCEIRAS DE 2007 A 201...,14695
15937,Poder Judiciário do Rio Grande do Norte3ª Turm...,Judicial,Intimação de Pauta,5º JFP da Com. Natal,10433,Certidão de decurso de prazo;Ato Ordinatório;R...,INICIAL ANO 2009 4ª VF INUNDAÇÃO ANTIGA NO JOS...,14695
15938,Poder Judiciário do Rio Grande do Norte3ª Turm...,Judicial,Intimação de Pauta,5º JFP da Com. Natal,9992,Certidão de decurso de prazo;Ato Ordinatório;R...,DOCUMENTO 01 - DECISAO CASO SIMILAR - OUTRO MA...,14695
15939,Poder Judiciário do Rio Grande do Norte3ª Turm...,Judicial,Intimação de Pauta,4º JFP da Com. Natal,9992,Contrarrazões;Recurso inominado - Município de...,OFÍCIO SEINFRA;1 - Ofício SEINFRA - 27.28 NOV ...,14695
